In [ ]:
!pip install kgtk

**Run the following cell, `gensim` is not installed with kgtk**

In [ ]:
!pip install gensim

  Using cached gensim-4.1.2-cp39-cp39-macosx_10_9_x86_64.whl (24.0 MB)
  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)


In [ ]:
import os
from kgtk.configure_kgtk_notebooks import ConfigureKGTK
from kgtk.functions import kgtk, kypher
from gensim.models import KeyedVectors
import tempfile
import h5py, torch
from torchbiggraph.model import ComplexDiagonalDynamicOperator, DotComparator, CosComparator
import json
import pandas as pd

/Users/amandeep/anaconda3/envs/kgtk-env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Parameters

# Folder on local machine where to create the output and temporary folders
input_path = None
output_path = "/tmp/projects"
project_name = "tutorial-graph-embeddings"

In [ ]:
files = [
    "all",
    "label",
    "alias",
    "description",
    "item",
    "qualifiers",
    "p31",
    "p279star"
]
additional_files = {
    'p31x': 'derived.P31x.tsv'
}
input_files_url = "https://github.com/usc-isi-i2/kgtk-tutorial-files/raw/main/datasets/arnold-profiled"
ck = ConfigureKGTK(files,
                  input_files_url=input_files_url)

ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name,
                 additional_files=additional_files)

In [ ]:
ck.print_env_variables()

KGTK_LABEL_FILE: /Users/amandeep/isi-kgtk-tutorial/tutorial-graph-embeddings_input/labels.en.tsv.gz
USE_CASES_DIR: /Users/amandeep/Github/kgtk-notebooks/use-cases
STORE: /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db
KGTK_OPTION_DEBUG: false
TEMP: /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings
OUT: /tmp/projects/tutorial-graph-embeddings
GRAPH: /Users/amandeep/isi-kgtk-tutorial/tutorial-graph-embeddings_input
kgtk: kgtk
kypher: kgtk query --graph-cache /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db
KGTK_GRAPH_CACHE: /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db
EXAMPLES_DIR: /Users/amandeep/Github/kgtk-notebooks/examples
all: /Users/amandeep/isi-kgtk-tutorial/tutorial-graph-embeddings_input/all.tsv.gz
label: /Users/amandeep/isi-kgtk-tutorial/tutorial-graph-embeddings_input/labels.en.tsv.gz
alias: /Users/amandeep/isi-kgtk-tutorial/t

In [ ]:
ck.load_files_into_cache()

kgtk query --graph-cache /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db -i "/Users/amandeep/isi-kgtk-tutorial/tutorial-graph-embeddings_input/all.tsv.gz" --as all  -i "/Users/amandeep/isi-kgtk-tutorial/tutorial-graph-embeddings_input/labels.en.tsv.gz" --as label  -i "/Users/amandeep/isi-kgtk-tutorial/tutorial-graph-embeddings_input/aliases.en.tsv.gz" --as alias  -i "/Users/amandeep/isi-kgtk-tutorial/tutorial-graph-embeddings_input/descriptions.en.tsv.gz" --as description  -i "/Users/amandeep/isi-kgtk-tutorial/tutorial-graph-embeddings_input/claims.wikibase-item.tsv.gz" --as item  -i "/Users/amandeep/isi-kgtk-tutorial/tutorial-graph-embeddings_input/qualifiers.tsv.gz" --as qualifiers  -i "/Users/amandeep/isi-kgtk-tutorial/tutorial-graph-embeddings_input/derived.P31.tsv.gz" --as p31  -i "/Users/amandeep/isi-kgtk-tutorial/tutorial-graph-embeddings_input/derived.P279star.tsv.gz" --as p279star  -i "/Users/amandeep/isi-kgtk-tutorial/tutorial-graph-

In [ ]:
vector_dimension = 100
vector_output_path = f"{os.environ['OUT']}/arnold.embeddings.augmented.{vector_dimension}.tsv"
vector_output_w2v_path = f"{os.environ['OUT']}/arnold.embeddings.augmented.{vector_dimension}.w2v.tsv"

## Compute ComplEx Graph Embeddings

In this notebook we will compute graph embeddings using `kgtk graph-embeddings` command for the `arnold` subgraph and demonstrate a few applications.

First step is to augment the `claims.wikibase-item.tsv.gz` file with `derived.P31x.tsv` file which contains occupations for humans as `instance of (P31)`

- `claims.wikibase-item.tsv.gz`: KGTK claims file non literal edges only
- `derived.P31x.tsv`: file with additional P31x links, adding occupation as `instance of` (computed)

In [ ]:
!kgtk cat -i $item \
-i $GRAPH/derived.P31x.tsv \
-o $GRAPH/claims.wikibase-item.augmented.tsv.gz

### Run `kgtk graph-embeddings`

The `kgtk graph-embeddings` command takes as input a KGTK edge file and computes graph embeddings of user specified type, producing vectors of user specified dimensions.

The following parameters are used in this instance:

- `-op ComplEx`: compute ComplEx graph embeddings
- `--dimension 30`: desired dimension of the vectors
- `-ot kgtk`: output format - kgtk
- `--retain_temporary_data True`: retain the byproduct files, which we will use in subsequent steps
- `-T <folder path>`: temporary folder where the temporary files will be stored
- `-i <file>`: input file
- `-o <file>`: output file
- `--log <file>`: log file

**NOTE: The graph embeddings command will take approximately 10 minutes to run**

In [ ]:
kgtk(f""" --debug graph-embeddings
            -op ComplEx 
            --dimension {vector_dimension}
            -ot kgtk
            --retain_temporary_data True
            -T $TEMP
            -w 1
            -i $GRAPH/claims.wikibase-item.augmented.tsv.gz
            -o {vector_output_path}
            --log $TEMP/ge.log.txt
    """)

#### Take a peek at the embeddings file.

In [ ]:
kgtk(f"""head -i {vector_output_path}""")

### The output is in `kgtk` format. Convert it to `word2vec` format for `gensim` similarity computation


For reference: 
- [gensim](https://radimrehurek.com/gensim/)
- [word2vec](https://en.wikipedia.org/wiki/Word2vec)

In [ ]:
kgtk(f"""convert-embeddings-format -i {vector_output_path}
                                   --input-property graph_embeddings
                                   --output-format word2vec
                                   -o {vector_output_w2v_path}
""")

### Load the vectors into `gensim`

To find similar vectors based on cosine similarity

In [ ]:
ge_vectors = KeyedVectors.load_word2vec_format(f"{vector_output_w2v_path}", binary=False)

Define a function to compute the `topn` similar vectors, and get the labels and descriptions of the matching Qnodes.

In [ ]:
def kgtk_most_similar(
    vectors,
    positive,
    relation_label="similarity_score",
    add_label_description=True,
    output_path=None,
    topn=25,
):
    """
    find topn similar Qnodes, add label and decription for the Qnodes
    
    :param vectors: vector space loaded into gensim KeyedVectors model
    :param positive: vector(s) or Qnode(s) to find similar entities for
    :param relation_label: name of the property to be used for the output file
    :param add_label_description: boolean parameter to add label and description for matched entities
    :param output_path: path to store the output file
    :param topn: desirednumber of similar entities
    """
    result = []
    if add_label_description:
        fp = tempfile.NamedTemporaryFile(
            mode="w", suffix=".tsv", delete=False, encoding="utf-8"
        )
        fp.write("node1\tlabel\tnode2\n")
        for (qnode, similarity) in vectors.most_similar(positive=positive, topn=topn):
            fp.write("{}\t{}\t{}\n".format(qnode, relation_label, similarity))
        filename = fp.name
        fp.close()

        os.environ["_temp_file"] = filename

        result = !$kypher -i label -i description -i "$_temp_file" --as sim \
--match 'sim: (n1)-[]->(similarity), label: (n1)-[]->(lab), description: (n1)-[]->(des)' \
--return 'distinct n1 as node1, similarity as node2, "similarity" as label, lab as `node1;label`, des as `node1;description`' \
--order-by 'cast(similarity, float) desc' 
        
        os.remove(filename)
        
    else:
        result.append("node1\tlabel\tnode2\n")
        for (qnode, similarity) in vectors.most_similar(positive=positive, topn=topn):
            result.append("{}\t{}\t{}\n".format(qnode, relation_label, similarity))

    if output_path:
        handle = open(output_path, "w")
        for line in result:
            handle.write(line)
            handle.write("\n")
        handle.close()
    else:
        columns = result[0].split("\t")
        data = []
        for line in result[1:]:
            data.append(line.split("\t"))
        return pd.DataFrame(data, columns=columns)

### Link Prediction

The following code reads the vectors for Qnodes as `head` and Properties as `relation`.

The files used in the code are produced by `kgtk graph-embeddings` code as a byproduct, in the folder specified by the `-T` option

In [ ]:
relation_names_list = json.load(open(f"{os.environ['TEMP']}/output/dynamic_rel_names.json"))
entity_names_list = json.load(open(f"{os.environ['TEMP']}/output/entity_names_all_0.json"))
prop_count = len(relation_names_list)

# operators
operator_lhs = ComplexDiagonalDynamicOperator(vector_dimension, prop_count)
operator_rhs = ComplexDiagonalDynamicOperator(vector_dimension, prop_count)
comparator = DotComparator()
cos_comparator = CosComparator()
with h5py.File(f"{os.environ['TEMP']}/output/model/model.v100.h5", "r") as hf:
    operator_state_dict_lhs = {
        "real": torch.from_numpy(hf["model/relations/0/operator/lhs/real"][...]),
        "imag": torch.from_numpy(hf["model/relations/0/operator/lhs/imag"][...]),
    }
    operator_state_dict_rhs = {
        "real": torch.from_numpy(hf["model/relations/0/operator/rhs/real"][...]),
        "imag": torch.from_numpy(hf["model/relations/0/operator/rhs/imag"][...]),
    }
    
operator_lhs.load_state_dict(operator_state_dict_lhs)
operator_rhs.load_state_dict(operator_state_dict_rhs)

# Load the embeddings
with h5py.File(f"{os.environ['TEMP']}/output/model/embeddings_all_0.v100.h5", "r") as hf:
    arnold_embedding = torch.from_numpy(hf["embeddings"][...])


entity_to_index = {}
for i, entity in enumerate(entity_names_list):
    entity_to_index[entity] = i
    

rel_index = {}
for i, rel in enumerate(relation_names_list):
    rel_index[rel] = i

The following function takes as input a `Qnode` and a `Property`, and outputs a vector which should be similar to the value of the relation.

For example, Qnode: `Q37079` = Tom Cruise, Property: `P166` = awards received and output a vector similar to awards. We will see this equation in action in the subsequent examples.

In [ ]:
def get_embed(head, relation=None):
    ''' This function generate the embeddings for the tail entities:
            Head entities: Obtained from the model
            Head + relation: Obtained using torch
        :param head: subject Qnode
        :param relation: optional property
    '''
    if relation is None:
        return arnold_embedding[entity_to_index[head], :].detach().numpy()
    return  operator_lhs(
                arnold_embedding[entity_to_index[head], :].view(1, vector_dimension),
                torch.tensor([rel_index[relation]])
            ).detach().numpy()[0]

#### Get the vector for `Q37079` (Tom Cruise) + `P166` (award received), then find most similar entities

In [ ]:
_vector = get_embed('Q37079', 'P166')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

#### Get the vector for `Q170564` (Terminator 2: Judgement Day) + `P161` (cast member), then find most similar entities

In [ ]:
_vector = get_embed('Q170564', 'P161')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

#### Get the vector for `Q104123` (Pulp Fiction) + `P161` (cast member), then find most similar entities

In [ ]:
_vector = get_embed('Q104123', 'P161')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

#### Get the vector for `Q2685` (Arnold Schwarzenegger), then find most similar entities

In [ ]:
_vector = get_embed('Q2685')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

#### Get the vector for `Q103148` (Lahn River), then find most similar entities

In [ ]:
_vector = get_embed('Q103148')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

## Prepare files for Google Projector

In this section, we will prepare `vectors` and `metadata` files for google projector.

We are focusing on the following types:

- `Q11424` (film)
- `Q33999` (actor)
- `Q4022` (river)
- `Q82955` (politician)

First step is to create a file with the following information ,

1. node1 :- Qnode
2. label :- name of the property
3. node2 :- embedding vector for node1
4. node1_label :- label for node1
5. type :- `instance of` for node1
6. type_label :- label for type

In [ ]:
%%time
kgtk(f""" query -i $GRAPH/claims.wikibase-item.augmented.tsv.gz 
         -i p279star 
         -i label 
         -i {vector_output_path} 
         -i $GRAPH/derived.P31x.tsv 
         --match 'item: (n1)-[]->(), 
             P31x: (n1)-[]->(c), 
             p279star: (c)-[]->(class), 
             label: (n1)-[]->(n1_label), 
             label: (class)-[]->(class_label), embeddings: (n1)-[l]->(embedding)'
        --where 'class in ["Q11424", "Q33999", "Q4022", "Q82955"]' 
        --return 'distinct n1, 
                  l.label as label,
                  embedding as node2,
                  kgtk_lqstring_text(n1_label) as `node1_label`, 
                  group_concat(distinct class) as type, 
                  group_concat(distinct kgtk_lqstring_text(class_label)) as `type_label`'
        -o $TEMP/arnold.embeddings.google.projector.tsv
""")

#### Take a peek at the file

In [ ]:
kgtk("""head -i $TEMP/arnold.embeddings.google.projector.tsv""")

#### Convert the `$TEMP/arnold.embeddings.google.projector.tsv` file to Google projector and metadata files

In [ ]:
kgtk(f"""--debug convert-embeddings-format 
          -i $TEMP/arnold.embeddings.google.projector.tsv
          --metadata-columns node1,node1_label,type,type_label
          --input-property graph_embeddings
          --metadata-file $OUT/arnold.metadata.{vector_dimension}.tsv
          --output-format gprojector
          -o $OUT/arnold.vectors.{vector_dimension}.tsv
""")

#### Peek at the metadata file

In [ ]:
kgtk(f"""head -i $OUT/arnold.metadata.{vector_dimension}.tsv""")

#### Peek at the vectors file

In [ ]:
kgtk(f"""head -i $OUT/arnold.vectors.{vector_dimension}.tsv""")tsv

### Download the files to local computer

**Run the following three cells only if you are running from Google Colab**

In [ ]:
from google.colab import files

#### Download the vectors file

In [ ]:
files.download(f"{os.environ['OUT']}/arnold.vectors.{vector_dimension}.tsv")

#### Download the metadata file

In [ ]:
files.download(f"{os.environ['OUT']}/arnold.metadata.{vector_dimension}.tsv")

**The files are now downloaded to your local 'Downloads' folder.**

## Google embedding projector


In a new tab, to go the URL: https://projector.tensorflow.org

Load the `vector` and `metadata` files using the load button
![Google embedding projector](https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/tutorial/assets/ge-load.png "Google embedding projector")

#### UMAP visualization of the embeddings

![Google embedding projector](https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/tutorial/assets/ge-arnold-umap.png "Google embedding projector")

#### PCA visualization of the embeddings, colored by `type_label`

![PCA Color by Type](https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/tutorial/assets/ge-arnold-pca.png "PCA Color by Type")